In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd 
import time

In [2]:
# this code is used to read the data from the csv file and then centralise as well as normalize the data

df=pd.read_csv('hand_landmarks_final.csv')
df2=df.copy()
df2=df2.drop(df2.columns[63],axis=1)
df5=df["63"]
x_coords = df2.iloc[:, ::3]
y_coords = df2.iloc[:, 1::3]

for i in range(len(df)):
    min_x_coords = x_coords.loc[i].min()
    min_x_index=int(x_coords.loc[i].idxmin())
    df3=df2.loc[i]
    min_y_coords=df3[min_x_index+1]
    # print(min_x_coords,min_y_coords)
    df2.iloc[i,::3]=df2.iloc[i,::3]-min_x_coords
    df2.iloc[i,1::3]=df2.iloc[i,1::3]-min_y_coords
df2

df4=df2.sub(df2.mean(axis=1), axis=0).div(df2.std(axis=1), axis=0)
df4["63"]=df5
df4

C:\Users\hgoel\AppData\Local\Temp\ipykernel_16400\48720169.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  min_y_coords=df3[min_x_index+1]
C:\Users\hgoel\AppData\Local\Temp\ipykernel_16400\48720169.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  min_y_coords=df3[min_x_index+1]
C:\Users\hgoel\AppData\Local\Temp\ipykernel_16400\48720169.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  min_y_coords=df3[min_x_index+1]
C:\U

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.057277,2.143830,-0.407538,1.121773,1.595695,-0.492090,1.931490,0.624711,-0.460194,2.473384,...,-0.169098,-1.268845,-0.895380,0.079614,-0.557816,-0.815411,0.164513,0.085659,-0.666978,A
1,-0.118506,2.278483,-0.487525,1.004239,1.902237,-0.531196,1.736534,0.957260,-0.484293,2.201850,...,-0.079870,-1.304730,-1.108118,0.199662,-0.602795,-1.098420,0.277032,0.105650,-0.989810,A
2,-0.241098,2.089003,-0.471362,1.024405,1.666187,-0.552053,1.910818,0.803082,-0.507014,2.568017,...,-0.268204,-1.308652,-0.826628,-0.053629,-0.582727,-0.724170,0.037463,0.056512,-0.563571,A
3,-0.257672,2.225779,-0.511756,0.920896,1.906840,-0.566848,1.738837,1.107225,-0.535359,2.313934,...,-0.153786,-1.419373,-1.050907,0.105251,-0.724826,-1.004716,0.150324,-0.007241,-0.880382,A
4,-0.344041,2.230485,-0.518106,1.008251,1.812100,-0.514465,1.848373,0.973722,-0.436687,2.449495,...,-0.204334,-1.409139,-0.949977,-0.001541,-0.620718,-0.866773,0.006950,0.011624,-0.722838,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,0.191461,1.175377,0.058039,0.783770,1.210527,-0.292767,1.470591,0.418290,-0.418625,1.733415,...,0.283946,-1.022683,-0.766026,0.153855,-0.347081,-0.677084,0.058040,0.058040,-0.515371,Z
1208,0.099648,1.267713,0.007401,0.740751,1.310884,-0.329644,1.449275,0.495115,-0.431530,1.725249,...,0.270962,-1.119711,-0.743031,0.118391,-0.445626,-0.653459,0.007402,0.007402,-0.487342,Z
1209,0.149269,1.003480,-0.052474,0.664898,1.317844,-0.386378,1.470237,0.587669,-0.454877,1.794422,...,0.337667,-1.140766,-0.771441,0.113233,-0.483283,-0.681749,-0.052473,-0.052473,-0.516706,Z
1210,-0.294333,1.850290,-0.330308,0.329167,2.022884,-0.694571,1.144277,1.174547,-0.789796,1.492257,...,0.058115,-0.496986,-1.135891,-0.103843,0.253389,-1.047933,-0.282210,0.731448,-0.862893,Z


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df4.iloc[:, :-1].astype(float)
y = df4.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.9836065573770492
Classification Report:
              precision    recall  f1-score   support

           A       1.00      0.75      0.86         4
           B       0.90      1.00      0.95         9
           C       1.00      1.00      1.00         7
           D       1.00      1.00      1.00         1
           E       1.00      1.00      1.00         6
           F       1.00      1.00      1.00         3
           G       1.00      1.00      1.00         6
           H       1.00      1.00      1.00         5
           I       1.00      1.00      1.00         7
           J       1.00      1.00      1.00         4
           K       0.50      1.00      0.67         1
           L       1.00      1.00      1.00         6
           M       1.00      1.00      1.00         5
           N       1.00      1.00      1.00         6
           O       1.00      1.00      1.00         5
           P       1.00      1.00      1.00         4
           Q       1.00      

In [4]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=50)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
report_dt = classification_report(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {accuracy_dt}")
print("Decision Tree Classification Report:")
print(report_dt)

Decision Tree Accuracy: 0.9262295081967213
Decision Tree Classification Report:
              precision    recall  f1-score   support

           A       1.00      0.75      0.86         4
           B       0.90      1.00      0.95         9
           C       1.00      1.00      1.00         7
           D       0.50      1.00      0.67         1
           E       1.00      1.00      1.00         6
           F       1.00      1.00      1.00         3
           G       1.00      0.83      0.91         6
           H       1.00      1.00      1.00         5
           I       1.00      1.00      1.00         7
           J       1.00      1.00      1.00         4
           K       1.00      1.00      1.00         1
           L       1.00      0.83      0.91         6
           M       1.00      0.60      0.75         5
           N       0.80      0.67      0.73         6
           O       1.00      1.00      1.00         5
           P       1.00      1.00      1.00         4
 

In [5]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=2)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print(f"KNN Accuracy: {accuracy_knn}")
print("KNN Classification Report:")
print(report_knn)

KNN Accuracy: 0.9836065573770492
KNN Classification Report:
              precision    recall  f1-score   support

           A       1.00      0.75      0.86         4
           B       0.90      1.00      0.95         9
           C       1.00      1.00      1.00         7
           D       1.00      1.00      1.00         1
           E       1.00      1.00      1.00         6
           F       1.00      1.00      1.00         3
           G       1.00      1.00      1.00         6
           H       1.00      1.00      1.00         5
           I       1.00      1.00      1.00         7
           J       1.00      1.00      1.00         4
           K       1.00      1.00      1.00         1
           L       1.00      1.00      1.00         6
           M       1.00      1.00      1.00         5
           N       1.00      1.00      1.00         6
           O       1.00      1.00      1.00         5
           P       1.00      1.00      1.00         4
           Q       1.

In [6]:
# this code loads the model and then uses the model to predict the sign language gestures

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            hand_landmarks_array_row = np.array([])

            for lm in hand_landmarks.landmark:
                hand_landmarks_array_row = np.append(hand_landmarks_array_row, [lm.x, lm.y, lm.z])

            # print(hand_landmarks_array_row)
            min_x_coords = hand_landmarks_array_row[0]
            min_y_coords = hand_landmarks_array_row[1]
            for i in range(0, len(hand_landmarks_array_row), 3):
                if hand_landmarks_array_row[i] < min_x_coords:
                    min_x_coords = hand_landmarks_array_row[i]
                    min_y_coords = hand_landmarks_array_row[i + 1]

            for i in range(0, len(hand_landmarks_array_row), 3):
                hand_landmarks_array_row[i] = hand_landmarks_array_row[i] - min_x_coords
                hand_landmarks_array_row[i + 1] = hand_landmarks_array_row[i + 1] - min_y_coords

            hand_landmarks_array_row = (hand_landmarks_array_row - np.mean(hand_landmarks_array_row)) / np.std(hand_landmarks_array_row)
            x_test=pd.DataFrame(hand_landmarks_array_row)   

            value = clf.predict(x_test.T )
            cv2.putText(frame, str(value), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Frame', frame)

    if cv2.waitKey(5) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\hgoel\Desktop\perceptohub_proj\sign_recog_try\sign_rec_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\hgoel\Desktop\perceptohub_proj\sign_recog_try\sign_rec_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\hgoel\Desktop\perceptohub_proj\sign_recog_try\sign_rec_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\hgoel\Desktop\perceptohub_proj\sign_recog_try\sign_rec_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\hgoel\Desktop\p